In [1]:
import streamlit as st
import pandas as pd
import mysql.connector
from datetime import datetime
from tqdm import tqdm
import csv
import pymysql
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time

def scrap_update():
    # 스크래핑 후 DB 각 테이블에 밀어넣기
    itching_url = 'https://www.coupang.com/np/search?rocketAll=false&searchId=5eeac5a9b93b4ed49d170607b2515921&q=%EA%B0%80%EB%A0%A4%EC%9B%80&brand=&offerCondition=&filter=&availableDeliveryFilter=&filterType=&isPriceRange=false&priceRange=&minPrice=&maxPrice=&page=1&trcid=&traid=&filterSetByUser=true&channel=user&backgroundColor=&searchProductCount=10394&component=&rating=0&sorter=saleCountDesc&listSize=36'
    atopy_url = 'https://www.coupang.com/np/search?rocketAll=false&searchId=9b262daefc044e919ab54daee9c54d21&q=%EC%95%84%ED%86%A0%ED%94%BC&brand=&offerCondition=&filter=&availableDeliveryFilter=&filterType=&isPriceRange=false&priceRange=&minPrice=&maxPrice=&page=1&trcid=&traid=&filterSetByUser=true&channel=user&backgroundColor=&searchProductCount=14363&component=&rating=0&sorter=saleCountDesc&listSize=36'
    psoriasis_url = 'https://www.coupang.com/np/search?rocketAll=false&searchId=861551980cb04aef840b28fd50528b8e&q=%EA%B1%B4%EC%84%A0&brand=&offerCondition=&filter=&availableDeliveryFilter=&filterType=&isPriceRange=false&priceRange=&minPrice=&maxPrice=&page=1&trcid=&traid=&filterSetByUser=true&channel=user&backgroundColor=&searchProductCount=3575&component=&rating=0&sorter=saleCountDesc&listSize=36'
    seborrheic_url = 'https://www.coupang.com/np/search?rocketAll=false&searchId=921c6c505c404156ae38158577502644&q=%EC%A7%80%EB%A3%A8%EC%84%B1+%ED%94%BC%EB%B6%80%EC%97%BC&brand=&offerCondition=&filter=&availableDeliveryFilter=&filterType=&isPriceRange=false&priceRange=&minPrice=&maxPrice=&page=1&trcid=&traid=&filterSetByUser=true&channel=user&backgroundColor=&searchProductCount=2953&component=&rating=0&sorter=saleCountDesc&listSize=36'
    eczema_url = 'https://www.coupang.com/np/search?rocketAll=false&searchId=8b7c65d2d6304667a7b77b702dbddb20&q=%EC%8A%B5%EC%A7%84&brand=&offerCondition=&filter=&availableDeliveryFilter=&filterType=&isPriceRange=false&priceRange=&minPrice=&maxPrice=&page=1&trcid=&traid=&filterSetByUser=true&channel=user&backgroundColor=&searchProductCount=5009&component=&rating=0&sorter=saleCountDesc&listSize=36'
    url_list = [itching_url, atopy_url, psoriasis_url, seborrheic_url, eczema_url]

    table_list = ['itching', 'atopy', 'psoriasis', 'seborrheic', 'eczema']

    for url, table in zip(url_list, table_list):
        print(f'{table}테이블 제품데이터 스크래핑 시작')

        prd_name_list = []
        prd_price_list = []
        prd_rate_list = []
        prd_link_list = []

        options = webdriver.ChromeOptions()
        options.add_argument('--disable-blink-features=AutomationControlled')
        driver = webdriver.Chrome()

        driver.get(url)
        time.sleep(1)

        prd_container = driver.find_element(By.ID, 'productList')
        prd_list = prd_container.find_elements(By.TAG_NAME, 'li')

        for idx, prd in enumerate(tqdm(prd_list, desc=f'{table}테이블 진행')):
            try:
                prd_name = prd.find_element(By.CSS_SELECTOR, '.name').text
                prd_price = int(prd.find_element(By.CSS_SELECTOR, '.price-value').text.replace(',', ''))
                prd_rate = float(prd.find_element(By.CSS_SELECTOR, '.rating').text)
                link = prd.find_element(By.CSS_SELECTOR, '.search-product-link')
                prd_link = link.get_attribute('href')
                           
                prd_name_list.append(prd_name)
                prd_price_list.append(prd_price)
                prd_rate_list.append(prd_rate)
                prd_link_list.append(prd_link)
            except:
                pass

        driver.quit()

        time.sleep(1)
        print(f'{table}테이블 제품 데이터 스크래핑 완료', '\n')

        data = {
            '제품명': prd_name_list,
            '가격': prd_price_list,
            '평점': prd_rate_list,
            '링크': prd_link_list,
        }
        
        conn, cur = None, None
        conn = pymysql.connect(host='localhost', user='root', password='1111', db='coupangDB', charset='utf8')
        cur = conn.cursor()

        cur.execute(f'DROP TABLE IF EXISTS {table}')
        cur.execute(f'CREATE TABLE IF NOT EXISTS {table} (id INT AUTO_INCREMENT PRIMARY KEY, product char(100), price INT, rate FLOAT, url TEXT)')

        sql = f'''INSERT INTO {table} (product, price, rate, url) VALUES (%s, %s, %s, %s)'''

        print(f'{table} TABLE INSERT...')
        for i in tqdm(range(len(data['제품명']))):
            product = data['제품명'][i]
            price = data['가격'][i]
            rate = data['평점'][i]
            url = data['링크'][i]

            cur.execute(sql, (product, price, rate, url))
            conn.commit()
            time.sleep(0.05)

        conn.close()
        print(f'{table} TABLE 업데이트가 완료되었습니다', '\n')
        print('=' * 50, '\n')
        time.sleep(0.5)

def CoupangDB_Update():
    date = datetime.today().strftime("%Y년 %m월 %d일")
    print(f'{date}자 coupangDB 업데이트를 시작합니다')
    print('=' * 50, '\n')
    scrap_update()

if __name__ == "__main__":
    CoupangDB_Update()

2024년 02월 05일자 coupangDB 업데이트를 시작합니다

itching테이블 제품데이터 스크래핑 시작


KeyboardInterrupt: 